# Generate Permeability Tables

This notebook generates the tables with the terrain permeability used in the paper. 
The permeability is obtained using the notebook [`RefineResistanceWithEbirdData`](RefineResistanceWithEbirdData.ipynb) using IUCN terrain compatibility as starting point, and eBird data to refine them. 
This notebook simply generates text tables that can be included in the paper. 

In [2]:
import pandas as pd
from birdmaps import bird_runs

In [3]:
FILE_PATH = "data/CA-Final"
bird_run = bird_runs.BirdRun(FILE_PATH)

acowoo_run = bird_run.get_bird_run("acowoo", "Acorn Woodpecker")
stejay_run = bird_run.get_bird_run("stejay", "Steller's Jay")

In [4]:
df_acowoo = pd.read_csv(acowoo_run.transmission_fn)
df_stejay = pd.read_csv(stejay_run.transmission_fn)

Produces dictionaries mapping code to transmission for the two species.

In [8]:
dict_acowoo = {}
for r in df_acowoo.iterrows():
    c = r[1][0]
    t = r[1][1]
    dict_acowoo[c] = t
dict_stejay = {}
for r in df_stejay.iterrows():
    c = r[1][0]
    t = r[1][1]
    dict_stejay[c] = t


Reads the names of the habitats

In [13]:
df_habitats = pd.read_csv("Habitat_codes_IUCN.csv")
df_habitats.columns

Index(['IUCNLevel', 'NewCode', 'Resistance forest bird', 'Unnamed: 3'], dtype='object')

In [19]:
dict_names = {}
for i in range(len(df_habitats["NewCode"])):
    try:
        dict_names[int(df_habitats["NewCode"][i])] = df_habitats["IUCNLevel"][i]
    except:
        pass

Creates a dataframe with map code, and transmission for the two birds.

In [36]:
nonzero_codes_acowoo = {int(k) for k, v in dict_acowoo.items() if v > 0.01}
nonzero_codes_stejay = {int(k) for k, v in dict_stejay.items() if v > 0.01}
nonzero_codes = list(sorted(list(nonzero_codes_acowoo | nonzero_codes_stejay)))
values_acowoo = [dict_acowoo.get(k, 0.) for k in nonzero_codes]
values_stejay = [dict_stejay.get(k, 0.) for k in nonzero_codes]
names = [dict_names.get(k, "---") for k in nonzero_codes]

In [39]:
df_permeability = pd.DataFrame({
    "Landcover Code": nonzero_codes,
    "Landcover Name": names,
    "Acorn Woodpecker": values_acowoo,
    "Steller's Jay": values_stejay,
})

In [40]:
df_permeability

,Terrain Code,Terrain Name,Acorn Woodpecker,Steller's Jay
0,104,1.4. Forest – Temperate,1.000000,1.000000
1,105,1.5. Forest – Subtropical/tropical dry,1.000000,0.900000
2,201,2.1. Savanna - Dry,0.033518,0.000431
3,304,3.4. Shrubland –Temperate,0.358599,0.323961
4,305,3.5. Shrubland – Subtropical/tropical dry,0.098745,0.052497
5,308,3.8. Shrubland – Mediterranean-type shrubby ve...,1.000000,0.212406
6,401,4.1. Grassland – Tundra,0.001869,0.015351
7,404,4.4. Grassland – Temperate,0.900000,0.040211
8,405,4.5. Grassland – Subtropical/tropical dry,0.900000,0.000862
9,500,5. Wetlands (inland),0.031533,0.016645


In [41]:
df_permeability.to_latex("Figures/Permeability.tex", index=False, float_format="%.3f")